**Generación de Notificaciones Falsas**

Para generar las direcciones aleatorias, usaremos archivos de acceso libre del Correo Uruguayo, disponible en https://catalogodatos.gub.uy/dataset/ide-direcciones-geograficas-del-uruguay 
Concatenamos todos los archivos en un único archivo csv. Eso permite que las direcciones sean geolocalizables y tengan sentido. De hecho, el archivo contiene las coordenadas de latitud y longitud de cada dirección. Al generar cada notificación, se eligirá una dirección al azar del archivo.


In [4]:
import pandas as pd
import os

# Lista de archivos CSV
archivos_csv = [
    'artigas.csv', 'canelones.csv', 'cerrolargo.csv', 'colonia.csv', 'durazno.csv',
    'flores.csv', 'florida.csv', 'lavalleja.csv', 'maldonado.csv', 'montevideo.csv',
    'paysandu.csv', 'rionegro.csv', 'rivera.csv', 'rocha.csv', 'salto.csv',
    'sanjose.csv', 'soriano.csv', 'tacuarembo.csv', 'treintaytres.csv'
]

# Leer y concatenar los archivos sin duplicar los títulos, usando low_memory=False
dataframes = [pd.read_csv(archivo, low_memory=False) for archivo in archivos_csv]
df_unido = pd.concat(dataframes, ignore_index=True)

# Guardar el DataFrame resultante a un nuevo archivo CSV
df_unido.to_csv('direcciones_de_uruguay.csv', index=False)


In [5]:
display(df_unido)

,latitud,longitud,punto_wkb,codigo_postal,codigo_via,nombre_via,num_puerta,letra_puerta,km,manzana,solar,nombre_inmueble,localidad,codigo_localidad,departamento
0,-30.393342,-56.466233,0101000020D17F0000C3F5285CD9D22041000000C0F051...,55000,27114.0,12 DE OCTUBRE,2.0,NaN,NaN,NaN,NaN,NaN,ARTIGAS,135.0,ARTIGAS
1,-30.399250,-56.458571,0101000020D17F00007B14AE4793D820411F85EB314C51...,55000,27114.0,12 DE OCTUBRE,6.0,NaN,NaN,NaN,NaN,NaN,ARTIGAS,135.0,ARTIGAS
2,-30.396084,-56.462343,0101000020D17F000000000000C2D520419A999959A451...,55000,27114.0,12 DE OCTUBRE,8.0,NaN,NaN,NaN,NaN,NaN,ARTIGAS,135.0,ARTIGAS
3,-30.393411,-56.466146,0101000020D17F0000A4703D0AEAD22041333333D3EE51...,55000,27114.0,12 DE OCTUBRE,10.0,NaN,NaN,NaN,NaN,NaN,ARTIGAS,135.0,ARTIGAS
4,-30.393215,-56.465923,0101000020D17F00008FC2F52815D320419A999939F451...,55000,27114.0,12 DE OCTUBRE,13.0,NaN,NaN,NaN,NaN,NaN,ARTIGAS,135.0,ARTIGAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486375,-33.408042,-54.608429,0101000020D17F0000713D0A57C40B2641EC51B8DE3D09...,30300,39648.0,VECINAL A ZAPICAN,9001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TREINTA Y TRES
1486376,-32.909574,-54.249243,0101000020D17F0000CDCCCCCC251C2741CDCCCC2C6D3E...,33000,16431.0,VECINAL LEONCHO,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TREINTA Y TRES
1486377,-32.952456,-53.517681,0101000020D17F00009A999919B131294133333393CD37...,31100,37782.0,ZAPATA L1,9000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TREINTA Y TRES
1486378,-32.877252,-53.607981,0101000020D17F00009A999919CEF12841CDCCCC4C3840...,31100,37763.0,ZAPATA L2,9000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TREINTA Y TRES


Ahora establecemos la lista de eventos notificables (solo elegimos unos pocos eventos como ejemplo de caso de uso) y la lista de prestadores de servicios médicos según el departamento.

In [10]:
# Listas para los eventos y prestadores
eventos = [
    "Dengue",
    "Leptospirosis",
    "Meningitis Viral",
    "Meningitis Bacteriana"]
prestadores_por_departamento = {
    'ARTIGAS': ["ASSE", "COMERI", "GREMEDA IAMPP"],
    'CANELONES': ["ASOCIACION ESPAÑOLA", "ASSE", "CAAMEPA IAMPP", "CASMU IAMPP", "CIRCULO CATOLICO", "COMECA IAMPP", "COSEM IAMPP", "CRAMI IAMPP", "CUDAM", "GREMCA", "HOSPITAL EVANGELICO", "MEDICA URUGUAYA", "SMI", "UNIVERSAL"],
    'CERRO LARGO': ["ASSE", "CAMCEL", "MEDICA URUGUAYA"],
    'COLONIA': ["ASSE", "CAMEC", "CAMOC IAMPP", "CIRCULO CATOLICO", "HOSPITAL EVANGELICO", "MEDICA URUGUAYA"],
    'DURAZNO': ["ASSE", "CAMEDUR IAMPP", "MEDICA URUGUAYA"],
    'FLORES': ["ASSE", "COMEFLO IAMPP"],
    'FLORIDA': ["ASSE", "CIRCULO CATOLICO", "COMEF IAMPP", "HOSPITAL EVANGELICO", "MEDICA URUGUAYA"],
    'LAVALLEJA': ["ASSE", "CAMDEL IAMPP", "IAC", "MEDICA URUGUAYA"],
    'MALDONADO': ["ASSE", "AMDM IAMPP", "ASOCIACION ESPAÑOLA", "BLUE CROSS", "CASMU IAMPP", "CIRCULO CATOLICO", "CRAME IAMPP", "MEDICA URUGUAYA"],
    'MONTEVIDEO': ["ASSE", "ASOCIACION ESPAÑOLA", "BLUE CROSS", "CASMU IAMPP", "CIRCULO CATOLICO", "COSEM IAMPP", "CUDAM", "GREMCA", "HOSPITAL BRITANICO", "HOSPITAL EVANGELICO", "MEDICARE", "MEDICA URUGUAYA", "MP", "SEGURO AMERICANO", "SMI", "SUMMUN", "UNIVERSAL"],
    'PAYSANDU': ["ASSE", "COMEPA"],
    'RIVERA': ["ASSE", "CASMER IAMPP", "COMERI", "MEDICA URUGUAYA"],
    'ROCHA': ["ASSE", "COMERO IAMPP", "MEDICA URUGUAYA"],
    'RIO NEGRO': ["ASSE", "AMEDRIN", "CAMY", "HOSPITAL EVANGELICO", "MEDICA URUGUAYA"],
    'SALTO': ["ASSE", "SMQS"],
    'SAN JOSE': ["ASSE", "AMSJ IAMPP", "ASOCIACION ESPAÑOLA", "CASMU IAMPP", "CIRCULO CATOLICO", "GREMCA", "HOSPITAL EVANGELICO", "MEDICA URUGUAYA"],
    'SORIANO': ["ASSE", "CAMS IAMPP", "CIRCULO CATOLICO", "HOSPITAL EVANGELICO", "MEDICA URUGUAYA"],
    'TACUAREMBO': ["ASSE", "COMTA IAMPP", "MEDICA URUGUAYA"],
    'TREINTA Y TRES': ["ASSE", "CAMCEL", "IAC", "MEDICA URUGUAYA"]
}

Generador de cédulas de identidad y características de teléfono por localidad

In [9]:
# URL de la API para generar cédulas
api_url = "https://ciuy-consumers-api.vercel.app/generate"

# Mapeo de números de teléfono por localidad/departamento

telefonos = {
    'CANELONES': '433x xxxx',
    'MALDONADO': '42xx xxxx',
    'ROCHA': '447x xxxx',
    'TREINTA y TRES': '445x xxxx',
    'CERRO LARGO': '464x xxxx',
    'RIVERA': '462x xxxx',
    'ARTIGAS': '477x xxxx',
    'SALTO': '473x xxxx',
    'PAYSANDU': '472x xxxx',
    'RIO NEGRO': '456x xxxx',
    'COLONIA': '452x xxxx',
    'SAN JOSE': '434x xxxx',
    'FLORES': '4364 xxxx',
    'FLORIDA': '435x xxxx',
    'LAVALLEJA': '444x xxxx',
    'DURAZNO': '436x xxxx',
    'SORIANO': '453x xxxx',
    'TACUAREMBO': '463x xxxx',
    'MONTEVIDEO': '2xxx xxxx'
}

Detalles adicionales según el evento

In [27]:
import random

def generar_informacion_adicional(evento):
    # Diccionario con la información adicional
    informacion_adicional = {
        'Dengue': {
            'Fiebre': ['S', 'N'],
            'Dolor abdominal intenso y continuo': ['S', 'N'],
            'Cefalea': ['S', 'N'],
            'Edemas': ['S', 'N'],
            'Dolor retroorbitario': ['S', 'N'],
            'Sangrado de Mucosas': ['S', 'N'],
            'Nauseas/Vómitos': ['S', 'N'],
            'Sangrado Grave': ['S', 'N'],
            'Exantema': ['S', 'N'],
            'Letargia o irritabilidad': ['S', 'N'],
            'Mialgias y/o artralgias': ['S', 'N'],
            'Shock': ['S', 'N'],
            'Hematocrito (%)': lambda: random.randint(20, 60),
            'Plaquetas (/mm3)': lambda: random.randint(30_000, 1_500_000),
            'Leucocitos sangre (/mm3)': lambda: random.randint(1_000, 40_000)
        },
        'Leptospirosis': {
            'Fiebre': ['S', 'N'],
            'Cefalea': ['S', 'N'],
            'Síndrome hemorragíparo': ['S', 'N'],
            'Mialgias y/o artralgias': ['S', 'N'],
            'Compromiso renal': ['S', 'N'],
            'Compromiso hepático': ['S', 'N'],
            'Sintomatología digestiva': ['S', 'N'],
            'Ictericia': ['S', 'N'],
            'Hiperemia conjuntival': ['S', 'N'],
            'Neumonia/Neumonitis': ['S', 'N'],
            'Lesiones de piel': ['S', 'N'],
            'Hematocrito (%)': lambda: random.randint(20, 60),
            'Plaquetas (/mm3)': lambda: random.randint(30_000, 1_500_000),
            'Leucocitos sangre (/mm3)': lambda: random.randint(1_000, 40_000),
            'Función hepática BT (mg/dl)': lambda: round(random.uniform(0.1, 12.0), 1),
            'Función hepática TGO (UI/l)': lambda: round(random.uniform(10, 200), 1),
            'Función hepática LDH (mg/dl)': lambda: random.randint(140, 1500),
            'Función hepática TGP (UI/l)': lambda: random.randint(7, 200),
            'Función renal azoemia (mg/dl)': lambda: round(random.uniform(7, 100), 1),
            'Función renal creatinemia (mg/dl)': lambda: round(random.uniform(0.6, 10.0), 1)
        },
        'Meningitis Viral': {
            'Fiebre': ['S', 'N'],
            'Convulsión': ['S', 'N'],
            'Petequias': ['S', 'N'],
            'Signos de irritación meníngea': ['S', 'N'],
            'Alteración de conciencia': ['S', 'N'],
            'Hipopefusión periférica': ['S', 'N'],
            'Paciente Grave': ['S', 'N'],
            'Directo LCR': ['Cocos Gram positivos', 'Cocos Gram negativos', 'Bacilos Gram positivos', 'Bacilos Gram negativos', 'No se observan bacterias'],
            'Aspecto LCR (citoquímico)': ['turbio', 'transparente'],
            'Glucosa LCR (mg/dl)': lambda: round(random.uniform(0, 99), 1),
            'Proteínas en LCR (mg/dl)': lambda: round(random.uniform(15, 50), 1),
            'Leucocitos LCR (/mm3)': lambda: random.randint(0, 10000),
            'Predominio': ['PMN', 'Linfocitos']
        },
        'Meningitis Bacteriana': {
            'Fiebre': ['S', 'N'],
            'Convulsión': ['S', 'N'],
            'Petequias': ['S', 'N'],
            'Signos de irritación meníngea': ['S', 'N'],
            'Alteración de conciencia': ['S', 'N'],
            'Hipopefusión periférica': ['S', 'N'],
            'Paciente Grave': ['S', 'N'],
            'Directo LCR': ['Cocos Gram positivos', 'Cocos Gram negativos', 'Bacilos Gram positivos', 'Bacilos Gram negativos', 'No se observan bacterias'],
            'Aspecto LCR (citoquímico)': ['turbio', 'transparente'],
            'Glucosa LCR (mg/dl)': lambda: round(random.uniform(0, 99), 1),
            'Proteínas en LCR (mg/dl)': lambda: round(random.uniform(15, 50), 1),
            'Leucocitos LCR (/mm3)': lambda: random.randint(0, 10000),
            'Predominio': ['PMN', 'Linfocitos']
        }
    }
    
    # Verificar si el evento es válido
    if evento not in informacion_adicional:
        raise ValueError(f"Evento '{evento}' no reconocido.")
    
    # Obtener los datos para el evento especificado
    info_evento = informacion_adicional[evento]
    
    # Generar valores para cada campo
    resultado = {}
    for clave, valor in info_evento.items():
        if callable(valor):
            resultado[clave] = valor()
        else:
            resultado[clave] = random.choice(valor)
    
    return resultado


Genero las notificaciones aleatorias usando faker

In [44]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta
import requests
import json

# Inicializar Faker

fake = Faker('es_ES')  # Faker para generar nombres y apellidos en español

# Función para calcular la edad a partir de la fecha de nacimiento y la fecha de inicio de síntomas

def calcular_edad(fecha_nacimiento, fecha_inicio_sintomas):
    edad = fecha_inicio_sintomas.year - fecha_nacimiento.year - ((fecha_inicio_sintomas.month, fecha_inicio_sintomas.day) < (fecha_nacimiento.month, fecha_nacimiento.day))
    return edad

# Generar un número celular

def generar_numero_celular():
    return f"09{random.randint(1000000, 9999999)}"

# Generar cédula uruguaya usando la API

def get_random_ci():
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Lanza un error si la solicitud falla
        data = response.json()
        return data.get("ci", None)  # Obtén el CI del JSON
    except requests.RequestException as e:
        print(f"Error al realizar la solicitud: {e}")
        return None

# Función para generar un número de teléfono basado en el departamento

def generar_telefono(departamento):
    depto = departamento.upper()   # Normaliza el nombre del departamento
    formato = telefonos.get(depto, 'N/A')  # Busca el formato en el diccionario
    if formato == 'N/A':  # Si no hay formato para el departamento, devuelve 'N/A'
        return 'N/A'
    # Genera un número reemplazando las 'x' por números aleatorios
    telefono = ''.join(str(random.randint(0, 9)) if char == 'x' else char for char in formato)
    return telefono

# Cargar el DataFrame de direcciones

direcciones_df = pd.read_csv("direcciones.csv")

# Definir fechas para el rango

fecha_inicio = datetime(2024, 1, 1)
fecha_fin = datetime(2024, 8, 9)

# Elegir prestador

def seleccionar_prestador(departamento):
    depto = departamento.upper()  # Convertir el nombre del departamento a mayúsculas
    prestadores = prestadores_por_departamento.get(depto, [])
    return random.choice(prestadores) if prestadores else 'N/A'


# Generar datos ficticios

num_personas = 500  # Número de personas a generar
personas = []

for _ in range(num_personas):
    # Determinar el sexo al azar
    sexo = random.choice(['M', 'F'])
    
    # Generar nombre y apellido según el sexo
    nombre = fake.first_name_male() if sexo == 'M' else fake.first_name_female()
    apellido = fake.last_name()
    
    # Seleccionar una dirección al azar
    direccion = direcciones_df.sample().iloc[0]
    calle = direccion['nombre_via']
    numero = direccion['num_puerta']
    
    # Asegurarse de que 'numero' sea una cadena, y convertirlo si es necesario
    numero_str = str(numero) if not pd.isna(numero) else ''
    
    # Concatenar calle y número
    if pd.notna(calle) and calle.strip():
        if numero_str.strip():
            address = f"{calle} {numero_str}"
        else:
            address = calle
    else:
        address = ''
    
    localidad = direccion['localidad']
    departamento = direccion['departamento']
    latitud = direccion['latitud']
    longitud = direccion['longitud']
    
    # Obtener teléfono y prestador
    telefono = generar_telefono(departamento)
    prestador = seleccionar_prestador(departamento)
    
    # Generar fechas
    fecha_nacimiento = fake.date_of_birth(minimum_age=0, maximum_age=90)
    fecha_inicio_sintomas = fake.date_between_dates(date_start=fecha_inicio, date_end=fecha_fin)
    
    # Calcular edad
    edad = calcular_edad(fecha_nacimiento, fecha_inicio_sintomas)
    
    # Seleccionar evento
    evento = random.choice(eventos)
    
    # Generar información adicional
    informacion_adicional = generar_informacion_adicional(evento)
    
    internado=random.choice(['S', 'N'])
    
    if internado =='S':
        tipo_internado=random.choice(['CTI', 'CM'])
    else:
        tipo_internado='sin dato'
    
    # Crear el diccionario de la persona
    persona = {
        "nombre": nombre,
        "apellido": apellido,
        "direccion": address,
        "localidad": localidad,
        "departamento": departamento,
        "latitud": latitud,
        "longitud": longitud,
        "cedula": get_random_ci(),
        "fecha_nacimiento": fecha_nacimiento.strftime('%Y-%m-%d'),
        "fecha_inicio_sintomas": fecha_inicio_sintomas.strftime('%Y-%m-%d'),
        "edad": edad,
        "sexo": sexo,
        "evento": evento,
        "telefono": telefono,
        "celular": generar_numero_celular(),
        "prestador": prestador,
        "internado": internado,
        "tipo_internado": tipo_internado,
        **informacion_adicional  # Agregar información adicional según el evento
    }
    
    personas.append(persona)


# Guardar los datos como JSON
with open("notificaciones_ficticias.json", "w") as f:
    json.dump(personas, f, indent=4)

In [49]:
import json

# Imprimir en formato JSON con indentación
print(json.dumps(personas, indent=4, ensure_ascii=False))


[
    {
        "nombre": "Luis Miguel",
        "apellido": "Guillén",
        "direccion": "SARANDI 10.0",
        "localidad": "SAN CARLOS",
        "departamento": "MALDONADO",
        "latitud": -34.7891206860802,
        "longitud": -54.9127554562374,
        "cedula": "87613478",
        "fecha_nacimiento": "1957-09-14",
        "fecha_inicio_sintomas": "2024-07-19",
        "edad": 66,
        "sexo": "M",
        "evento": "Meningitis Viral",
        "telefono": "4211 5854",
        "celular": "096987669",
        "prestador": "MEDICA URUGUAYA",
        "internado": "S",
        "tipo_internado": "CM",
        "Fiebre": "N",
        "Convulsión": "N",
        "Petequias": "N",
        "Signos de irritación meníngea": "N",
        "Alteración de conciencia": "N",
        "Hipopefusión periférica": "S",
        "Paciente Grave": "N",
        "Directo LCR": "Bacilos Gram negativos",
        "Aspecto LCR (citoquímico)": "transparente",
        "Glucosa LCR (mg/dl)": 46.9,
        

Ahora genero los datos del laboratorio, a partir de 450 de las 500 notificaciones generadas.


In [66]:
import json
import pandas as pd
import random
from datetime import datetime, timedelta

# Leer el archivo JSON completo si contiene un array de objetos
def leer_json_completo(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        try:
            objetos = json.load(file)
        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON: {e}")
            return []
    return objetos

# Ruta al archivo JSON
archivo_json = 'notificaciones_ficticias.json'

# Leer los objetos JSON del archivo
personas = leer_json_completo(archivo_json)

# Convertir lista de personas a DataFrame
df_personas = pd.DataFrame(personas)

# Seleccionar 450 registros únicos por CI
selected_cis = df_personas['cedula'].unique()
selected_personas = df_personas[df_personas['cedula'].isin(random.sample(list(selected_cis), min(450, len(selected_cis))))]

# Generar DataFrame de laboratorio
laboratorio_data = []

# Separar personas con evento "Dengue" y el resto
dengue_personas = selected_personas[selected_personas['evento'] == 'Dengue']
otros_personas = selected_personas[selected_personas['evento'] != 'Dengue']

# Procesar personas con evento "Dengue" (95% resultados positivos)
for _, persona in dengue_personas.iterrows():
    fecha_muestra = datetime.strptime(persona['fecha_inicio_sintomas'], '%Y-%m-%d') + timedelta(days=5)
    estado = 'finalizado'  # Para asegurar que todos los resultados positivos tengan estado finalizado
    test = random.choice(['serología', 'PCR'])
    resultado = 'positivo' if random.random() < 0.95 else 'negativo'
    
    laboratorio_data.append({
        'cedula': persona['cedula'],
        'nombre': persona['nombre'],
        'apellido': persona['apellido'],
        'evento': persona['evento'],
        'fecha_muestra': fecha_muestra.strftime('%Y-%m-%d'),
        'estado': estado,
        'test': test,
        'resultado': resultado
    })

# Procesar el resto de las personas (50% resultados positivos)
for _, persona in otros_personas.iterrows():
    fecha_muestra = datetime.strptime(persona['fecha_inicio_sintomas'], '%Y-%m-%d') + timedelta(days=5)
    estado = random.choice(['en tránsito', 'recibido', 'en proceso', 'finalizado'])
    
    # Si el estado no es 'finalizado', el resultado debe estar vacío
    if estado == 'finalizado':
        resultado = 'positivo' if random.random() < 0.50 else 'negativo'
    else:
        resultado = None
    
    # Si el resultado es positivo, asegurarse de que el estado sea 'finalizado'
    if resultado == 'positivo':
        estado = 'finalizado'
    
    laboratorio_data.append({
        'cedula': persona['cedula'],
        'nombre': persona['nombre'],
        'apellido': persona['apellido'],
        'evento': persona['evento'],
        'fecha_muestra': fecha_muestra.strftime('%Y-%m-%d'),
        'estado': estado,
        'test': random.choice(['serología', 'PCR']),
        'resultado': resultado
    })

# Convertir lista a DataFrame
df_laboratorio = pd.DataFrame(laboratorio_data)

# Guardar el DataFrame de laboratorio en un archivo CSV con las columnas deseadas
df_laboratorio.to_csv('lab-data.csv', columns=['cedula', 'nombre', 'apellido', 'evento', 'fecha_muestra', 'estado', 'test', 'resultado'], index=False)


In [67]:
display(df_laboratorio)

,cedula,nombre,apellido,evento,fecha_muestra,estado,test,resultado
0,11654303,Pepito,Malo,Dengue,2024-03-14,finalizado,PCR,positivo
1,99349801,Lidia,Canales,Dengue,2024-01-24,finalizado,PCR,positivo
2,10431065,Eloísa,Salazar,Dengue,2024-07-06,finalizado,serología,positivo
3,40582816,Carlos,Peñas,Dengue,2024-03-31,finalizado,PCR,positivo
4,57631591,Raúl,Comas,Dengue,2024-05-19,finalizado,serología,positivo
...,...,...,...,...,...,...,...,...
445,87598527,Ruben,Cervera,Meningitis Viral,2024-03-04,en proceso,serología,None
446,15111654,Manu,Sevilla,Leptospirosis,2024-08-05,en proceso,PCR,None
447,78659790,Brunilda,Alvarez,Meningitis Viral,2024-05-26,recibido,serología,None
448,26923791,Julia,Soler,Meningitis Bacteriana,2024-01-13,en proceso,serología,None


Ahora genero 5 años de datos de leptospirosis para calcular el corredor endémico

In [69]:
import json
import random
from faker import Faker
from datetime import datetime, timedelta
from decimal import Decimal

# Inicializar Faker
fake = Faker('es_ES')  # Faker para generar nombres y apellidos en español

# Función para convertir Decimal a float en un diccionario
def convertir_decimal_a_float(d):
    if isinstance(d, dict):
        return {k: convertir_decimal_a_float(v) for k, v in d.items()}
    elif isinstance(d, list):
        return [convertir_decimal_a_float(i) for i in d]
    elif isinstance(d, Decimal):
        return float(d)
    else:
        return d

# Función para calcular la semana epidemiológica (SE) a partir de la fecha de inicio de síntomas
def calcular_se(fecha_inicio, ae):
    fechas_rango = {
        2019: (datetime(2018, 12, 31), datetime(2019, 12, 28)),
        2020: (datetime(2019, 12, 29), datetime(2020, 12, 27)),
        2021: (datetime(2020, 12, 28), datetime(2021, 12, 26)),
        2022: (datetime(2021, 12, 27), datetime(2022, 12, 31)),
        2023: (datetime(2022, 12, 31), datetime(2023, 12, 30)),
    }
    
    start_date, end_date = fechas_rango[ae]
    days_diff = (fecha_inicio - start_date).days
    se = (days_diff // 7) + 1
    
    if ae == 2020 and se > 53:
        se = 53
    
    return se

# Generar datos ficticios de leptospirosis para un año epidemiológico específico
def generar_datos_leptospirosis(ae):
    leptospirosis_data = []
    
    fechas_rango = {
        2019: (datetime(2018, 12, 31), datetime(2019, 12, 28)),
        2020: (datetime(2019, 12, 29), datetime(2020, 12, 27)),
        2021: (datetime(2020, 12, 28), datetime(2021, 12, 26)),
        2022: (datetime(2021, 12, 27), datetime(2022, 12, 31)),
        2023: (datetime(2022, 12, 31), datetime(2023, 12, 30)),
    }
    
    start_date, end_date = fechas_rango[ae]
    num_notificaciones = random.randint(100, 130)
    
    for _ in range(num_notificaciones):
        fecha_inicio_sintomas = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
        se = calcular_se(fecha_inicio_sintomas, ae)
        fecha_muestra = fecha_inicio_sintomas + timedelta(days=5)
        
        leptospirosis_data.append({
            'nombre': fake.first_name(),
            'apellido': fake.last_name(),
            'direccion': f"{fake.street_name()} {random.randint(1, 100)}",
            'localidad': fake.city(),
            'departamento': fake.state(),
            'latitud': fake.latitude(),
            'longitud': fake.longitude(),
            'cedula': fake.ssn(),
            'fecha_nacimiento': fake.date_of_birth(minimum_age=0, maximum_age=90).strftime('%Y-%m-%d'),
            'fecha_inicio_sintomas': fecha_inicio_sintomas.strftime('%Y-%m-%d'),
            'edad': random.randint(0, 90),
            'sexo': random.choice(['M', 'F']),
            'evento': 'Leptospirosis',
            'telefono': fake.phone_number(),
            'celular': f"09{random.randint(1000000, 9999999)}",
            'prestador': fake.company(),
            'internado': random.choice(['S', 'N']),
            'tipo_internado': random.choice(['CTI', 'CM']) if random.choice(['S', 'N']) == 'S' else 'sin dato',
            'Fiebre': random.choice(['S', 'N']),
            'Cefalea': random.choice(['S', 'N']),
            'Síndrome hemorragíparo': random.choice(['S', 'N']),
            'Mialgias y/o artralgias': random.choice(['S', 'N']),
            'Compromiso renal': random.choice(['S', 'N']),
            'Compromiso hepático': random.choice(['S', 'N']),
            'Sintomatología digestiva': random.choice(['S', 'N']),
            'Ictericia': random.choice(['S', 'N']),
            'Hiperemia conjuntival': random.choice(['S', 'N']),
            'Neumonia/Neumonitis': random.choice(['S', 'N']),
            'Lesiones de piel': random.choice(['S', 'N']),
            'Hematocrito (%)': random.randint(30, 50),
            'Plaquetas (/mm3)': random.randint(100000, 1000000),
            'Leucocitos sangre (/mm3)': random.randint(10000, 30000),
            'Función hepática BT (mg/dl)': round(random.uniform(1.0, 20.0), 1),
            'Función hepática TGO (UI/l)': round(random.uniform(10.0, 100.0), 1),
            'Función hepática LDH (mg/dl)': random.randint(100, 1000),
            'Función hepática TGP (UI/l)': round(random.uniform(10.0, 100.0), 1),
            'Función renal azoemia (mg/dl)': round(random.uniform(10.0, 100.0), 1),
            'Función renal creatinemia (mg/dl)': round(random.uniform(0.5, 10.0), 1),
            'AE': ae,
            'SE': se,
            'estado': 'confirmado',
            'resultados_laboratorio': {
                'cedula': fake.ssn(),
                'nombre': fake.first_name(),
                'apellido': fake.last_name(),
                'evento': 'Leptospirosis',
                'fecha_muestra': fecha_muestra.strftime('%Y-%m-%d'),
                'estado': 'finalizado',
                'test': 'PCR',
                'resultado': 'positivo'
            }
        })
    
    return leptospirosis_data

# Generar datos para los años epidemiológicos entre 2019 y 2023
leptospirosis_data_total = []

for ae in range(2019, 2024):
    leptospirosis_data_total.extend(generar_datos_leptospirosis(ae))

# Convertir Decimals a floats
leptospirosis_data_total = convertir_decimal_a_float(leptospirosis_data_total)

# Guardar los datos en un archivo JSON
with open('leptospirosis_data.json', 'w', encoding='utf-8') as file:
    json.dump(leptospirosis_data_total, file, ensure_ascii=False, indent=4)

print("Datos de leptospirosis guardados en 'leptospirosis_data.json'.")


Datos de leptospirosis guardados en 'leptospirosis_data.json'.


In [70]:
import json

# Ruta al archivo JSON
archivo_json = 'leptospirosis_data.json'

# Cargar el archivo JSON
with open(archivo_json, 'r', encoding='utf-8') as archivo:
    datos = json.load(archivo)

# Mostrar los datos en un formato legible
for registro in datos:
    print(json.dumps(registro, indent=4, ensure_ascii=False))


{
    "nombre": "Zaira",
    "apellido": "Pascual",
    "direccion": "Avenida Anna Vidal 71",
    "localidad": "León",
    "departamento": "Huelva",
    "latitud": 53.0158835,
    "longitud": 57.081236,
    "cedula": "052-19-1434",
    "fecha_nacimiento": "1943-12-10",
    "fecha_inicio_sintomas": "2019-08-20",
    "edad": 41,
    "sexo": "F",
    "evento": "Leptospirosis",
    "telefono": "+34 947 446 068",
    "celular": "097026194",
    "prestador": "Berrocal y asociados S.Coop.",
    "internado": "S",
    "tipo_internado": "sin dato",
    "Fiebre": "N",
    "Cefalea": "N",
    "Síndrome hemorragíparo": "S",
    "Mialgias y/o artralgias": "N",
    "Compromiso renal": "N",
    "Compromiso hepático": "S",
    "Sintomatología digestiva": "N",
    "Ictericia": "S",
    "Hiperemia conjuntival": "N",
    "Neumonia/Neumonitis": "N",
    "Lesiones de piel": "N",
    "Hematocrito (%)": 43,
    "Plaquetas (/mm3)": 938524,
    "Leucocitos sangre (/mm3)": 10554,
    "Función hepática BT (mg/dl)

In [71]:
import json
from collections import Counter

# Ruta al archivo JSON
archivo_json = 'leptospirosis_data.json'

# Cargar el archivo JSON
with open(archivo_json, 'r', encoding='utf-8') as archivo:
    datos = json.load(archivo)

# Contar los casos por AE
contar_ae = Counter(registro.get('AE') for registro in datos)

# Mostrar el conteo por AE
for ae, conteo in contar_ae.items():
    print(f"Año Epidemiológico {ae}: {conteo} casos")


Año Epidemiológico 2019: 125 casos
Año Epidemiológico 2020: 130 casos
Año Epidemiológico 2021: 122 casos
Año Epidemiológico 2022: 113 casos
Año Epidemiológico 2023: 130 casos


In [73]:
import json
from collections import defaultdict

# Cargar los datos desde el archivo JSON
with open('leptospirosis_data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Inicializar un diccionario para contar los casos por SE
casos_por_se = defaultdict(int)

# Procesar los datos
for caso in data:
    if caso['AE'] == 2019:  # Filtrar solo los casos del año 2019
        se = caso['SE']
        casos_por_se[se] += 1  # Contar el caso para la semana epidemiológica correspondiente

# Convertir el resultado en una lista de diccionarios para facilitar la visualización
casos_por_se_list = [{'SE': se, 'cantidad_casos': count} for se, count in casos_por_se.items()]

# Mostrar los resultados
for entry in casos_por_se_list:
    print(f"Semana Epidemiológica {entry['SE']}: {entry['cantidad_casos']} casos")

# Guardar los resultados en un archivo JSON
with open('casos_por_se_2019.json', 'w', encoding='utf-8') as file:
    json.dump(casos_por_se_list, file, ensure_ascii=False, indent=4)


Semana Epidemiológica 34: 4 casos
Semana Epidemiológica 33: 1 casos
Semana Epidemiológica 31: 4 casos
Semana Epidemiológica 15: 2 casos
Semana Epidemiológica 37: 2 casos
Semana Epidemiológica 45: 1 casos
Semana Epidemiológica 32: 4 casos
Semana Epidemiológica 38: 2 casos
Semana Epidemiológica 42: 2 casos
Semana Epidemiológica 26: 3 casos
Semana Epidemiológica 35: 1 casos
Semana Epidemiológica 22: 4 casos
Semana Epidemiológica 1: 6 casos
Semana Epidemiológica 3: 4 casos
Semana Epidemiológica 51: 1 casos
Semana Epidemiológica 46: 5 casos
Semana Epidemiológica 52: 1 casos
Semana Epidemiológica 29: 4 casos
Semana Epidemiológica 36: 2 casos
Semana Epidemiológica 23: 3 casos
Semana Epidemiológica 40: 5 casos
Semana Epidemiológica 12: 2 casos
Semana Epidemiológica 28: 2 casos
Semana Epidemiológica 18: 3 casos
Semana Epidemiológica 14: 2 casos
Semana Epidemiológica 11: 5 casos
Semana Epidemiológica 25: 5 casos
Semana Epidemiológica 19: 2 casos
Semana Epidemiológica 2: 1 casos
Semana Epidemioló